In [1]:
from common import *
from search.imports import *
from search.search import *
from search.manager import ModelManager

In [2]:
M = pd.read_pickle(the_data/'given/M.pkl')

In [3]:
F,P = pd.read_pickle(big_data/'saves/train_5fixedsince+1.32.pkl')

### Search Params specs

In [4]:
class LSP(O()):
    class Discrete(O()):
        stop = {1:.45, 2:.5, 3:.55, 5:.6, 7:.65, 9:.7} #{1:.3, 3:.5, 5:.6, 7:.65}
        obo = {0: 0}
        enc = {
            'learning_rate': [.05],
            ('max_depth','num_leaves'):
                [(6,1<<6),(8,1<<8),(10,1<<10),(12,1<<12),
                 (10,1<<6),(-1,1<<10),(10,1<<8),(-1,1<<14),
                 (12,1<<10),(8,1<<6),(-1,1<<12),(12,1<<8),]
        }
        
    class OneByOne(O()):
        base = .33
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 0
        enc = {#TODO implement 1 sided search e.g. len(a)==2
            0: {
                'min_data_in_leaf': O(a=[1,60,375], b=120, cast=round, lim=lambda lim: lim),
                'min_sum_hessian_in_leaf': O(a=[0,50,200], lim=lambda lim: lim+1),
                'lambda_l1': O(a=[0,.02,.2], b=0),
                'lambda_l2': O(a=[0,.02,.2], b=0),
            }
        }

### Search Features/Samples specs

In [5]:
from features import gen_features

ho = P.quarter>=2015.5
ho.name = None


class LSM(O()):
    model = O(
        time = 'time',
        value = 'y',
        target = 'target',
        weight = 'flat_weight',
    )
    
    metric = KaggleMetric()
    search = LSP
    
    t_offset = 0
    num_feats = 10000
    num_samps = 100
    stop_samps = {2:.4, 3:.46, 4:.51, 5:.55, 6:.59, 7:.62, 8:float('inf')} #{1:.57, 2:.61, 3:.64, 4:float('inf')}
    
    
@inside(LSM)
class Step(metaclass=staticclass):
    _best = -float('inf')
    def func(results, *, t):
        t += 1
        self = __class__
        num_feats, num_samps, stop_samps = LSM.num_feats, LSM.num_samps, LSM.stop_samps

        if results is not None and results['score'] > self._best:
            self._best = results['score']

        ts = t % num_samps
        if ts == 0: # if new feature, reset `self._best` score
            self._best = -float('inf')
        assert 0 not in stop_samps
        if ts in stop_samps and stop_samps[ts] > self._best:
            t = (t//num_samps + 1) * num_samps # go to next feature if met stop score criterion

        if t >= num_feats * num_samps:
            return None
        return t

@inside(LSM)
class Features(metaclass=staticclass):
    '''features selection groups'''
    def func(*, t):
        t += LSM.t_offset
        num_feats, num_samps = LSM.num_feats, LSM.num_samps
        f, q = gen_features(t // num_samps)
        return O(Feats=f, Ctor=q)

@inside(LSM)
class Samples(metaclass=staticclass):
    '''sample learning/cv split'''
    # O(method='GroupShuffleSplit.2', kwargs=dict(n_splits=<(5)many>, test_size=.5, random_state=44), groups='quarter'),
    def ctor2samps(ctor):
        a, b = tuple(ctor)
        group = P.quarter[~ho]
        tr, cv = group.isin(a), group.isin(b)
        tr, cv = [F.index[~ho][trcv] for trcv in [tr,cv]]
        return ((tr, cv),(cv,tr))
    
    def func(*, t):
        t += LSM.t_offset
        group = P.quarter[~ho]
        h = frozenset(group)
        g = frozenset(random.Random(t).sample(h, len(h)//2))
        return O(Samps=__class__.ctor2samps((g,h-g)), Ctor=(g,h-g))

@inside(LSM, name='Params')
class Params(O()):
    '''parameters constant settings'''
    data = dict(
        objective = 'binary',
        num_iterations = 100000,
        early_stopping_round = 32,
        metric = 'None',
        seed = 44,
        bagging_seed = 45,
        feature_fraction_seed = 46,
    )

# LIVE TESTING WALK ONLY

In [12]:
# @REAL
DISPLAY = True

DIR = Path(top_dir/'data/search/flat')
DIR.mkdir(exist_ok=True)

TAG = 'flat'

LOG_DIR = DIR/'runs'
LOG_DIR.mkdir(exist_ok=True)
LOG = logger(file=LOG_DIR/(TAG+'.log'))
SEE = LOG

import globals as top_imports
top_imports.SEE = LOG.logger(display=DISPLAY)
#top_imports.SEE = lambda*a,**k:None

VERBOSE_EVAL = 50

In [13]:
#from search.LSM import LSM
mm = ModelManager(DIR)
ms = ModelSearch(specs=LSM, mm=mm, X=F, Y=P, tag=TAG, log=LOG.logger('>>>', display=DISPLAY), verbose_eval=VERBOSE_EVAL)
walk = ms.walk()

In [ ]:
%%time
for (fi, si), results in walk:
    for a in ['nboost', 'train', 'scores', 'score']:
        SEE(f'{a} = {results[a]}')